## I

### Carregando os dados

In [ ]:
import pandas as pd
import seaborn as sns

passageiros = pd.read_csv('Passageiros.csv')

sns.lineplot(x='tempo',y='passageiros', data=passageiros,label='dado_completo') 

### Escalando os dados

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(passageiros)
dado_escalado = sc.transform(passageiros)

x=dado_escalado[:,0] #Features - Características - Tempo
y=dado_escalado[:,1] #Alvo - Número de passageiros

sns.lineplot(x=x,y=y,label='dado_escalado') 
plt.ylabel('Passageiros')
plt.xlabel('Data')

### Dividindo em treino e teste

In [ ]:
tamanho_treino = int(len(passageiros)*0.9) #Pegando 90% dos dados para treino
tamanho_teste = len(passageiros)-tamanho_treino #O resto vamos reservar para teste

In [ ]:
xtreino = x[0:tamanho_treino]
ytreino = y[0:tamanho_treino]

In [ ]:
xteste = x[tamanho_treino:len(passageiros)]
yteste = y[tamanho_treino:len(passageiros)]

In [ ]:
sns.lineplot(x=xtreino,y=ytreino,label='treino') 
sns.lineplot(x=xteste,y=yteste,label='teste') 

## II

### Regressão Linear

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

regressor = Sequential()

In [ ]:
regressor = Sequential()
regressor.add(Dense(1, input_dim=1, kernel_initializer='Ones', activation='linear',use_bias=False))
regressor.compile(loss='mean_squared_error',optimizer='adam')
regressor.summary()

In [ ]:
regressor.fit(xtreino,ytreino)

In [ ]:
y_predict= regressor.predict(xtreino) #Prevendo os dados de treino (o ajuste)

In [ ]:
sns.lineplot(x=xtreino,y=ytreino,label='treino')
sns.lineplot(x=xtreino,y=y_predict[:,0],label='ajuste_treino')

In [ ]:
d = {'tempo': xtreino, 'passageiros': y_predict[:,0]}
resultados = pd.DataFrame(data=d)
resultados.head()

In [ ]:
resultado_transf = sc.inverse_transform(resultados)
resultado_transf = pd.DataFrame(resultado_transf)
resultado_transf.columns = ['tempo','passageiros']

sns.lineplot(x="tempo",y="passageiros",data=passageiros)
sns.lineplot(x="tempo",y="passageiros",data=resultado_transf)

In [ ]:
y_predict_teste= regressor.predict(xteste) #Prevendo os dados de teste(o futuro)

In [ ]:
d = {'tempo': xteste, 'passageiros': y_predict_teste[:,0]}
resultados_teste = pd.DataFrame(data=d)

In [ ]:
resultado_transf_teste = sc.inverse_transform(resultados_teste)

In [ ]:
resultado_transf_teste = pd.DataFrame(resultado_transf_teste)
resultado_transf_teste.columns = ['tempo','passageiros']

In [ ]:
sns.lineplot(x="tempo",y="passageiros",data=passageiros,label='dado_completo')
sns.lineplot(x="tempo",y="passageiros",data=resultado_transf,label='ajuste_treino')
sns.lineplot(x="tempo",y="passageiros",data=resultado_transf_teste,label='previsão')

### Regressão não-linear

In [ ]:
regressor2 = Sequential()

In [ ]:
regressor2.add(Dense(8, input_dim=1, kernel_initializer='random_uniform', activation='sigmoid',use_bias=False)) 
regressor2.add(Dense(8, kernel_initializer='random_uniform', activation='sigmoid',use_bias=False))
regressor2.add(Dense(1, kernel_initializer='random_uniform', activation='linear',use_bias=False))

In [ ]:
regressor2.compile(loss='mean_squared_error',optimizer='adam')
regressor2.summary()

In [ ]:
regressor2.fit(xtreino,ytreino,epochs =500)

In [ ]:
y_predict= regressor2.predict(xtreino) #Prevendo os dados de treino (o ajuste)

In [ ]:
y_predict_teste= regressor2.predict(xteste) #Prevendo os dados de teste(o futuro)

In [ ]:
sns.lineplot(x=xtreino,y=ytreino,label='treino')
sns.lineplot(x=xteste,y=yteste,label='teste')
sns.lineplot(x=xtreino,y=y_predict[:,0],label='ajuste_treino')
sns.lineplot(x=xteste,y=y_predict_teste[:,0],label='previsão')

## III

### Alterando a forma como passamos os dados

Agora x e y vão valores diferentes. X vai conter o número de passageiros em um tempo anterior e y vai conter o número de passageiros em t+1, por exemplo. 

In [ ]:
import numpy as np
vetor = pd.DataFrame(ytreino)[0]

In [ ]:
def separa_dados(vetor,n_passos):
  """Entrada: vetor: número de passageiros
               n_passos: número de passos no regressor
     Saída:
              X_novo: Array 2D 
              y_novo: Array 1D - Nosso alvo
  """
  X_novo, y_novo = [], []
  for i in range(n_passos,vetor.shape[0]):
    X_novo.append(list(vetor.loc[i-n_passos:i-1]))
    y_novo.append(vetor.loc[i])
  X_novo, y_novo = np.array(X_novo), np.array(y_novo) 
  return X_novo, y_novo

In [ ]:
xtreino_novo, ytreino_novo = separa_dados(vetor,1)

In [ ]:
xtreino_novo[0:5] #X

In [ ]:
ytreino_novo[0:5] #y

### Agora vamos separar o teste

In [ ]:
vetor2 = pd.DataFrame(yteste)[0]
xteste_novo, yteste_novo = separa_dados(vetor2,1)

### Voltando para as redes neurais

In [ ]:
regressor3 = Sequential()

regressor3.add(Dense(8, input_dim=1, kernel_initializer='ones', activation='linear',use_bias=False))
regressor3.add(Dense(64, kernel_initializer='random_uniform', activation='sigmoid',use_bias=False))
regressor3.add(Dense(1, kernel_initializer='random_uniform', activation='linear',use_bias=False))
regressor3.compile(loss='mean_squared_error',optimizer='adam')
regressor3.summary()

In [ ]:
regressor3.fit(xtreino_novo,ytreino_novo,epochs =100)

In [ ]:
y_predict_novo = regressor3.predict(xtreino_novo)

In [ ]:
sns.lineplot(x='tempo',y=ytreino_novo,data=passageiros[1:129],label='treino')
sns.lineplot(x='tempo',y=pd.DataFrame(y_predict_novo)[0],data=passageiros[1:129],label='ajuste_treino')

In [ ]:
y_predict_teste_novo = regressor3.predict(xteste_novo)

In [ ]:
resultado = pd.DataFrame(y_predict_teste_novo)[0]

sns.lineplot(x='tempo',y=ytreino_novo,data=passageiros[1:129],label='treino')
sns.lineplot(x='tempo',y=pd.DataFrame(y_predict_novo)[0],data=passageiros[1:129],label='ajuste_treino')

sns.lineplot(x='tempo',y=yteste_novo,data=passageiros[130:144],label='teste')
sns.lineplot(x='tempo',y=resultado.values,data=passageiros[130:144],label='previsão')

### Janelas

In [ ]:
xtreino_novo, ytreino_novo = separa_dados(vetor,4)

In [ ]:
xtreino_novo[0:5] #X

In [ ]:
ytreino_novo[0:5] #y

In [ ]:
xteste_novo, yteste_novo = separa_dados(vetor2,4)

In [ ]:
regressor4 = Sequential()

regressor4.add(Dense(8, input_dim=4, kernel_initializer='random_uniform', activation='linear',use_bias=False)) #relu
regressor4.add(Dense(64, kernel_initializer='random_uniform', activation='sigmoid',use_bias=False)) #relu
regressor4.add(Dense(1, kernel_initializer='random_uniform', activation='linear',use_bias=False))
regressor4.compile(loss='mean_squared_error',optimizer='adam')
regressor4.summary()

In [ ]:
regressor4.fit(xtreino_novo,ytreino_novo,epochs =300)

In [ ]:
y_predict_teste_novo = regressor4.predict(xteste_novo)

In [ ]:
resultado = pd.DataFrame(y_predict_teste_novo)[0]

sns.lineplot(x='tempo',y=ytreino_novo,data=passageiros[4:129],label='treino')
sns.lineplot(x='tempo',y=pd.DataFrame(y_predict_novo)[0],data=passageiros[4:129],label='ajuste_treino')

sns.lineplot(x='tempo',y=yteste_novo,data=passageiros[133:144],label='teste')
sns.lineplot(x='tempo',y=resultado.values,data=passageiros[133:144],label='previsão')

## IV

### LSTMs

In [ ]:
#Nova base de dados

bike = pd.read_csv('bicicletas.csv')
bike.head()

In [ ]:
bike['datas'] = pd.to_datetime(bike['datas'])
sns.lineplot(x='datas',y='contagem', data=bike)
plt.xticks(rotation=70)

### Escalando os dados

In [ ]:
sc2 = StandardScaler()
sc2.fit(bike['contagem'].values.reshape(-1,1))
y = sc2.transform(bike['contagem'].values.reshape(-1,1))

### Dividindo em treino e teste

In [ ]:
tamanho_treino = int(len(bike)*0.9) #Pegando 90% dos dados para treino
tamanho_teste = len(bike)-tamanho_treino #O resto vamos reservar para teste

In [ ]:
ytreino = y[0:tamanho_treino]
yteste = y[tamanho_treino:len(bike)]

In [ ]:
sns.lineplot(x='datas',y=ytreino[:,0],data=bike[0:tamanho_treino],label='treino') 
sns.lineplot(x='datas',y=yteste[:,0], data=bike[tamanho_treino:len(bike)],label='teste')
plt.xticks(rotation=70) 

In [ ]:
vetor = pd.DataFrame(ytreino)[0]
xtreino_novo, ytreino_novo = separa_dados(vetor,10)

In [ ]:
xtreino_novo[0:5]

In [ ]:
ytreino_novo[0:5]

In [ ]:
vetor2 = pd.DataFrame(yteste)[0]

In [ ]:
xteste_novo, yteste_novo = separa_dados(vetor2,10)

### O que a LSTM espera

A entrada de redes recorrentes deve possuir a seguinte forma para a entrada (número de amostras, número de passos no tempo, e número de atributos por passo no tempo).

In [ ]:
xtreino_novo = xtreino_novo.reshape((xtreino_novo.shape[0],xtreino_novo.shape[1],1))

In [ ]:
xtreino_novo.shape

In [ ]:
xteste_novo = xteste_novo.reshape((xteste_novo.shape[0],xteste_novo.shape[1],1))

### Usando a LSTM

In [ ]:
from tensorflow.keras.layers import LSTM

recorrente = Sequential()
recorrente.add(LSTM(128, input_shape=(xtreino_novo.shape[1],xtreino_novo.shape[2])))
recorrente.add(Dense(units=1))

In [ ]:
recorrente.compile(loss='mean_squared_error',optimizer='RMSProp')
recorrente.summary()

In [ ]:
resultado = recorrente.fit(xtreino_novo,ytreino_novo,validation_data=(xteste_novo,yteste_novo),epochs=100)

In [ ]:
y_ajustado = recorrente.predict(xtreino_novo)

In [ ]:
sns.lineplot(x='datas',y=ytreino[:,0],data=bike[0:tamanho_treino],label='treino')
sns.lineplot(x='datas',y=y_ajustado[:,0],data=bike[0:15662],label='ajuste_treino')
plt.xticks(rotation=70) 

In [ ]:
y_predito = recorrente.predict(xteste_novo)

In [ ]:
sns.lineplot(x='datas',y=yteste[:,0], data=bike[tamanho_treino:len(bike)],label='teste')
sns.lineplot(x='datas',y=y_predito[:,0], data=bike[tamanho_treino+10:len(bike)],marker='.',label='previsão')
plt.xticks(rotation=70) 

## V

In [ ]:
from tensorflow.keras.layers import GRU

recorrente_g = Sequential()
recorrente_g.add(GRU(128, input_shape=(xtreino_novo.shape[1],xtreino_novo.shape[2])))
recorrente_g.add(Dense(units=1))

In [ ]:
recorrente_g.compile(loss='mean_squared_error',optimizer='RMSProp')
recorrente_g.summary()

In [ ]:
resultado2 = recorrente_g.fit(xtreino_novo,ytreino_novo, validation_data=(xteste_novo,yteste_novo),epochs=100)

In [ ]:
y_predito2 = recorrente_g.predict(xteste_novo)

In [ ]:
sns.lineplot(x='datas',y=yteste[:,0], data=bike[tamanho_treino:len(bike)])
sns.lineplot(x='datas',y=y_predito2[:,0], data=bike[tamanho_treino+10:len(bike)],marker='.')
plt.legend(['conhecido','estimado'])
plt.xticks(rotation=70) 

In [ ]:
print(resultado2.history.keys())

In [ ]:
plt.plot(resultado.history['loss'])
plt.plot(resultado.history['val_loss'])
plt.legend(['treino','teste'])

In [ ]:
plt.plot(resultado2.history['loss'])
plt.plot(resultado2.history['val_loss'])
plt.legend(['treino','teste'])
